<a href="https://colab.research.google.com/github/BrunoSDomingues/projeto-iris/blob/main/projeto_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de detecção de íris

## Limpando o ambiente

In [1]:
!rm -rf G6_iris_recognition/
!rm -rf enhanced_imgs/
!rm -rf images/
!rm -rf model.pickle
!rm -rf *.bmp

## Baixando bibliotecas e imagens a serem utilizadas

In [2]:
!pip3 install scikit-image --upgrade

In [3]:
!git clone https://github.com/lucianosilva-github/G6_iris_recognition.git

Cloning into 'G6_iris_recognition'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 31 (delta 3), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [4]:
!git clone https://github.com/lucianosilva-github/images.git
!rm -rf images/.git/

Cloning into 'images'...
remote: Enumerating objects: 1266, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1266 (delta 1), reused 0 (delta 0), pack-reused 1246
Receiving objects: 100% (1266/1266), 189.65 MiB | 23.63 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (1163/1163), done.


In [5]:
# Imports
import cv2
import os
import G6_iris_recognition as g6
import matplotlib.pyplot as plt
import numpy as np

from google.colab.patches import cv2_imshow

## Tratando as imagens

### Melhorando a nitidez

Para melhorar a nitidez, podemos utilizar o conceito de convolução. Para tal, iremos aplicar uma "máscara" na imagem original utilizando a função [`filter2D`](https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04) do `cv2`. Esta máscara é chamada de [kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)), e segue o formato abaixo:

$\begin{bmatrix}
0 & -1 & 0\\
-1 & I & -1\\
0 & -1 & 0
\end{bmatrix}$

onde $I$ é a intensidade da nitidez. Geralmente utiliza-se $I = 5$, mas pode-se utilizar valores diferentes.

In [6]:
# Funcao que aplica a convolucao na imagem
def sharpen_image(img, intensity=5):
    """
    img = imagem de entrada
    intensity = intensidade do sharpen
    """
    kernel = np.array([[0, -1, 0], 
                       [-1, intensity, -1], 
                       [0, -1, 0]])
    
    # O parametro ddepth tem valor -1 para utilizar a mesma
    # quantidade de canais que a imagem original
    # https://docs.opencv.org/4.5.3/d4/d86/group__imgproc__filter.html#filter_depths
    sharpened = cv2.filter2D(src=img, ddepth=-1, kernel=kernel)

    return sharpened

### Melhorando o brilho e o contraste

De acordo com a [documentação oficial](https://docs.opencv.org/master/d3/dc1/tutorial_basic_linear_transform.html) do OpenCV, a melhor forma de fazer ajustes no brilho e no contraste é utilizando a fórmula abaixo

$g(x) = \alpha \cdot f(x) + \beta$

onde: 
- $\alpha$ é o parâmetro de _gain_ (utilizado para controlar o contraste);
- $\beta$ é o parâmetro de _bias_ (utilizado para controlar o brilho);

Como estamos tratando de uma imagem, a conta de ajuste deve ser feita para cada pixel. Assim, podemos adaptar a fórmula:

$img_{out} (i, j) = \alpha \cdot img_{in} (i, j) + \beta$

Para ajustes no contraste, basta alterar o valor de $\alpha$:
- para $\alpha = 1$, a imagem não tem alteração de contraste;
- para $0 < \alpha < 1$, o contraste da imagem é reduzido;
- para $\alpha > 1$, o contraste da imagem é intensificado;

Para ajustes no brilho, basta alterar o valor de $\beta$:
- para $\beta = 0$, a imagem não tem alteração de brilho;
- para $\beta < 0$, o brilho da imagem é reduzido;
- para $\beta > 0$, o brilho da imagem é intensificado;

De modo a fazer esta conta, devemos iterar por cada pixel e armazenar o valor calculado em uma nova matriz de imagem. Para tal, podemos usar a função [`convertScaleAbs`](hhttps://docs.opencv.org/4.5.3/d2/de8/group__core__array.html#ga3460e9c9f37b563ab9dd550c4d8c4e7d) do `cv2`, que faz a conta automaticamente.

In [7]:
# Funcao que ajusta simultaneamente o brilho e o contraste
def process_bc(img, alpha=1, beta=0, gamma=1):
    """
    img = imagem de entrada
    alpha = valor de contraste
    beta = valor do brilho
    """
    
    alpha = float(alpha)
    beta = int(beta)
    gamma = float(gamma)
    
    # Se o ajuste de contraste for 1 e o ajuste de brilho for 0, a imagem sera inalterada
    if alpha == 1 and beta == 0:
        return img
    
    else:
        if alpha < 0:
            raise ValueError("Alpha value can't be lower than 0")
        
        else:
            new_img = np.zeros(img.shape, img.dtype)
            new_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
                        
    # Gamma correction
    if gamma == 1:
        return new_img
    
    else:
        lookUpTable = np.empty((1,256), np.uint8)
        
        for i in range(256):
            lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
            
        corrected = cv2.LUT(new_img, lookUpTable)
        
        return corrected

In [8]:
"""
A célula abaixo foi usada para gerar as imagens utilizadas na comparação entre 
o processamento sem equalização do histograma de cores e o processamento com 
equalização do histograma de cores.
Para executar, basta trocar o parâmetro generate para True.
"""

generate = False

if generate:
    # Lendo a imagem inicial
    img = cv2.imread("images/0000/0000_000.bmp")

    # Convertendo em niveis de cinza
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Equalizando
    equalized = cv2.equalizeHist(img_gray)

    # Tratando
    sharpened_eq = sharpen_image(equalized, intensity = 5)
    processed_eq = process_bc(sharpened_eq, alpha = 1.2, beta = 10, gamma = 0.45)

    # Tratando sem equalizacao
    sharpened = sharpen_image(img_gray, intensity = 5)
    processed = process_bc(sharpened, alpha = 1.2, beta = 10, gamma = 0.45)

    cv2.imwrite("with_eq.bmp", processed_eq)
    cv2.imwrite("without_eq.bmp", processed)

### Equalização do histograma de cores

Algumas imagens podem ter cores que se destacam mais que outras. Para ajudar a melhor processar a imagem, pode-se aplicar uma equalização no histograma de cores da imagem, usando a função [`equalizeHist`](https://docs.opencv.org/3.4.15/d6/dc7/group__imgproc__hist.html#ga7e54091f0c937d49bf84152a16f76d6e) do `cv2`.

As imagens abaixo mostram a diferença entre uma imagem que teve seu histograma equalizado e posteriormente foi tratada e uma imagem que foi tratada sem equalizar o histograma. É possível notar que a equalização ajuda a destacar mais a diferenciar a esclerótica da íris, e evita um brilho excessivo.

Com equalização            |  Sem equalização
:-------------------------:|:-------------------------:
![](https://raw.githubusercontent.com/BrunoSDomingues/projeto-iris/main/with_eq.bmp)           |![](https://raw.githubusercontent.com/BrunoSDomingues/projeto-iris/main/without_eq.bmp)

## Tratando as imagens de entrada

In [9]:
# Gerando pasta para salvar as imagens tratadas
if not os.path.isdir("enhanced_imgs"):
    os.mkdir("enhanced_imgs")

In [10]:
# Gerando lista com os numeros das pastas
img_list = [f"{str(i).zfill(4)}" for i in range(60) if i != 12] # Nao existe pasta 0012

# A maior parte das pastas contém 20 imagens cada
n_imgs = 20

In [15]:
# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    # Checa se a pasta nao existe, e cria ela se nao existir
    if not os.path.isdir(enhanced_path):
        os.mkdir(enhanced_path)
        
    # Percorrendo cada imagem dentro da pasta
    for i in range(n_imgs):
        # Obtem o path completo da imagem
        filename = f"{path}_{str(i).zfill(3)}.bmp"
        img_path = os.path.join("images", path, filename)        
        
        # Existem casos em que uma pasta nao possui 20 imagens
        # Verifica-se portanto se a imagem existe antes de processar
        if not os.path.isfile(img_path):
            pass
        
        else:
            # Lendo a imagem original
            img = cv2.imread(img_path)

            # Convertendo em niveis de cinza
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Equalizando
            equalized = cv2.equalizeHist(img_gray)

            # Tratando
            sharpened = sharpen_image(equalized, intensity = 5)
            processed = process_bc(sharpened, alpha = 1.5, beta = 30, gamma = 1.25)

            # Convertendo de niveis de cinza para BGR
            img_final = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)

            # Salvando a imagem
            img_path = os.path.join(enhanced_path, filename)
            cv2.imwrite(img_path, img_final)

## Treinando o modelo

In [16]:
open("model.pickle", "a").close()
g6.iris_model_train("enhanced_imgs", "model.pickle")

directory_list ['enhanced_imgs/0028', 'enhanced_imgs/0047', 'enhanced_imgs/0009', 'enhanced_imgs/0046', 'enhanced_imgs/0026', 'enhanced_imgs/0033', 'enhanced_imgs/0023', 'enhanced_imgs/0041', 'enhanced_imgs/0018', 'enhanced_imgs/0057', 'enhanced_imgs/0014', 'enhanced_imgs/0005', 'enhanced_imgs/0019', 'enhanced_imgs/0049', 'enhanced_imgs/0037', 'enhanced_imgs/0002', 'enhanced_imgs/0022', 'enhanced_imgs/0039', 'enhanced_imgs/0017', 'enhanced_imgs/0003', 'enhanced_imgs/0008', 'enhanced_imgs/0054', 'enhanced_imgs/0031', 'enhanced_imgs/0038', 'enhanced_imgs/0052', 'enhanced_imgs/0015', 'enhanced_imgs/0058', 'enhanced_imgs/0011', 'enhanced_imgs/0007', 'enhanced_imgs/0036', 'enhanced_imgs/0034', 'enhanced_imgs/0000', 'enhanced_imgs/0006', 'enhanced_imgs/0013', 'enhanced_imgs/0044', 'enhanced_imgs/0040', 'enhanced_imgs/0030', 'enhanced_imgs/0056', 'enhanced_imgs/0051', 'enhanced_imgs/0053', 'enhanced_imgs/0004', 'enhanced_imgs/0016', 'enhanced_imgs/0029', 'enhanced_imgs/0043', 'enhanced_imgs/0

['0026', '0038', '0044', '0004', '0010']

## Testando o modelo

In [17]:
# Contadores
total_imgs = 0
matches = 0
failed = 0

# Matriz de confusao
confusion_matrix = []

# Percorrendo cada pasta
for path in img_list:
    enhanced_path = os.path.join("enhanced_imgs", path)
    
    for image in os.listdir(enhanced_path):
        # Contando o numero de imagens total
        total_imgs += 1
        
        # Pegando o path completo do arquivo
        img_path = os.path.join(enhanced_path, image)
        
        # Passando a imagem para o modulo G6
        guess = g6.iris_model_test("model.pickle", img_path)
        temp = guess
        
        # Se nao deu match, o valor na matriz de confusao deve ser -1
        if guess == "unmatch":
            failed += 1
            temp = -1
        
        # Se der match, o guess tem que bater com o numero da pasta
        elif int(guess) == int(path):
            matches += 1
        
        confusion_matrix.append(np.array([int(path), temp]))
        
confusion_matrix = np.array(confusion_matrix)

locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 local variable 'west_mark' referenced before assignment
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
locate expression 1 index 640 is out of bounds for axis 1 with size 640
rectangle expression2 not enough values to unpack (expected 2, got 1)
lo

In [18]:
print(f"Matches: {matches}")
print(f"Failures: {failed}")
print(f"Success rate: {round(matches/total_imgs*100, 2)}%")

Matches: 23
Failures: 1140
Success rate: 1.98%


In [21]:
# Escrevendo a matriz de confusao
np.savetxt("confusion_matrix.txt", confusion_matrix, fmt='%s')

## Análise dos resultados

Embora as imagens tenham sido tratadas para deixar a íris mais destacada, o modelo teve uma taxa de sucesso próxima de 2%, o que não é muito desejável. 

O principal fator que explica este resultado é que o módulo `G6_iris_recognition` tem como requerimento que no mínimo 90% da íris esteja visível, e a maior parte das imagens contém a íris parcialmente oculta, seja pelas pálpebras ou por cílios que estão na frente da íris.

![](https://github.com/BrunoSDomingues/projeto-iris/blob/main/requirements.png?raw=1)

Outro fator que pode ser considerado relevante é que as pálpebras refletiram muito a luz do flash das fotos originais, e ao realizar o tratamento este brilho ficou destacado.

Por fim, algo a ser notado é que as imagens originais vieram em preto e branco, quando na verdade poderiam ser coloridas e depois convertidas em níveis de cinza durante o tratamento.